In [2]:
from tensorflow import keras
import tensorflow as tf

In [7]:
def ConvBlock(x, n_filters):
    x = keras.layers.Conv2D(n_filters, 3, padding='same', activation='relu', kernel_initializer='he_normal')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)

    return x

def Encoder(x, filters):
    skips = []

    for f in filters:
        x = ConvBlock(x, f)
        x = ConvBlock(x, f)
        # 맨 마지막 층을 제외하고는 skip connection, downsampling을 진행
        if f != 1024:
            skips.append(x)
            x = keras.layers.MaxPooling2D(2)(x)
        
    return x, skips

def Decoder(x, filters, skips):
    for f, skip in zip(filters, skips):
        x = keras.layers.Conv2DTranspose(f, (2, 2), strides=2, padding='same')(x)
        x = ConvBlock(x, f)
        x = keras.layers.Concatenate()([x, skip]) 
        x = ConvBlock(x, f)
      
    x = ConvBlock(x, 2)
    x = keras.layers.Conv2D(filters=1, kernel_size=1, padding='same', activation='linear')(x)

    return x

def Unet(img_size):
    inputs = keras.Input(shape=img_size + (1,))

    # 축소 경로
    filters = [64, 128, 256, 512, 1024]

    x, skips = Encoder(inputs, filters)

    # 확장 경로
    x = Decoder(x, filters[::-1][1:], skips[::-1])

    model = keras.Model(inputs, x)

    return model

In [3]:
# Encoder만 정의하기
# filters = [64, 128, 256, 512, 1024]
# inputs = keras.Input(shape=(512, 512) + (1,))
# x = Encoder(inputs, filters)
# model = keras.Model(inputs, x)
# model.summary()

In [4]:
# model = Unet((512, 512))
# model.summary()

In [12]:
[(1, 2) for _ in range(3)][0][0]

1

In [5]:
def SMD_Unet(img_size):
    inputs = keras.Input(shape=img_size + (1,))

    # 축소 경로
    filters = [64, 128, 256, 512, 1024]

    x, skips = Encoder(inputs, filters)

    # 확장 경로
    # mask : HardExudate, Hemohedge, Microane, SoftExudates
    input_hat = Decoder(x, filters[::-1][1:], skips[::-1]) # 원본 이미지 추정
    ex = Decoder(x, filters[::-1][1:], skips[::-1])
    he = Decoder(x, filters[::-1][1:], skips[::-1])
    ma = Decoder(x, filters[::-1][1:], skips[::-1])
    se = Decoder(x, filters[::-1][1:], skips[::-1])

    model = keras.Model(inputs, outputs=[input_hat, ex, he, ma, se])

    return model

In [6]:
model = SMD_Unet((512, 512))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 64  640         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

fit 함수 정의해서 사용해야돼서 class api로 변경

In [13]:

import tensorflow as tf
from tensorflow import keras

class ConvBlock(tf.keras.layers.Layer):
    def __init__(self, n_filters):
        super(ConvBlock, self).__init__()
        self.filters = n_filters
        self.conv = keras.layers.Conv2D(n_filters, 3, padding='same', activation='relu', kernel_initializer='he_normal')
        self.bn = keras.layers.BatchNormalization()
        self.relu = keras.layers.Activation('relu')
    
    def call(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        
        return x

    
class UpsampleBlock(tf.keras.layers.Layer):
    def __init__(self, n_filters):
    super(UpsampleBlock, self).__init__()
    self.filters = n_filters
    self.conv_T = keras.layers.Conv2DTranspose(n_filters, (2, 2), strides=2, padding='same')
    self.conv_block_1 = ConvBlock(n_filters)
    self.concat = keras.layers.Concatenate()
    self.conv_block_2 = ConvBlock(n_filters)
  
  def call(self, x, skip):
    x = self.conv_T(x)
    x = self.conv_block_1(x)
    x = self.concat([x, skip])
    x = self.conv_block_2(x)
    
    return x   

class EncoderBlock(tf.keras.layers.Layer):
    def __init__(self, filters):
        super(EncoderBlock, self).__init__()
        self.filters = filters # filters = [n_filters]
        self.conv_blocks = [(ConvBlock(f), ConvBlock(f)) for f in self.filters]
        self.max_poolings = [keras.layers.MaxPooling2D(2) for _ in range(len(self.filters)-1)]
        # conv_block, max_pooling 리스트 길이 맞춰주기 위한 트릭
        self.max_poolings += [0]

    def call(self, x):
        skips = []
        for conv_block, max_pooling in zip(self.conv_blocks, self.max_poolings):
            x = conv_block[0](x)
            x = conv_block[1](x)

            # 맨 마지막 층을 제외하고는 skip connection, downsampling을 진행
            if conv_block[0].filters != 1024:
                skips.append(x)
                x = max_pooling(x)
        return x, skips

class DecoderBlock(tf.keras.layers.Layer):
    def __init__(self, filters):
        super(DecoderBlock, self).__init__()
        self.filters = filters # filters = [n_filters]
        self.upsample_blocks = [UpsampleBlock(f) for f in self.filters]
        self.last_block = tf.keras.Sequential([ConvBlock(2), 
                                               keras.layers.Conv2D(filters=1, kernel_size=1, padding='same', activation='linear')])

    def call(self, x, skips):
        for skip, upsample_block in zip(skips, self.upsample_blocks):
            x = upsample_block(x, skip)
        
        x = self.last_block(x)
        
        return x
      
class SMD_Unet(tf.keras.Model):
    def __init__(self):
        super(SMD_Unet, self).__init__()

        self.filters = [64, 128, 256, 512, 1024]

        # Encoder
        self.encoder = EncoderBlock(self.filters)

        # Decoder
        # 5종류의 decoder가 있음
        # reconstruction
        # HardExudate, Hemohedge, Microane, SoftExudates
        self.reconstruction = DecoderBlock(self.filters[::-1][1:])
        self.HardExudate = DecoderBlock(self.filters[::-1][1:])
        self.Hemohedge = DecoderBlock(self.filters[::-1][1:])
        self.Microane = DecoderBlock(self.filters[::-1][1:])
        self.SoftExudates = DecoderBlock(self.filters[::-1][1:])


    def call(self, inputs, only_recons=False):
        # Encoder
        x, skips = self.encoder(inputs)
        # Decoder
        input_hat = self.reconstruction(x, skips[::-1])
        # reconstruction만 학습할 때 구분
        if only_recons:     
            return [input_hat]
        else:
            ex = self.HardExudate(x, skips[::-1])
            he = self.Hemohedge(x, skips[::-1])
            ma = self.Microane(x, skips[::-1])
            se = self.SoftExudates(x, skips[::-1])
            
            return [input_hat, ex, he, ma, se]

In [40]:
from data_generator import DR_Generator

In [109]:
import os

masks = ['HardExudate_Masks', 'Hemohedge_Masks', 'Microaneurysms_Masks', 'SoftExudate_Masks']
mask_dir = '../data/FGADR-Seg-set_Release/Seg-set'
mask_paths = [os.path.join(mask_dir, mask) for mask in masks]

generator_args = {
  'dir_path':'../data/FGADR-Seg-set_Release/Seg-set/Original_Images/',
  'mask_path':mask_paths,
  'use_mask':True,
  'img_size':(512, 512),  
  'batch_size':4, # 8로 하면 바로 OOM 뜸
  'dataset':'FGADR', # FGADR or EyePacks
  'is_train':True
}

tr_eyepacks_gen = DR_Generator(start_end_index=(0, 500), **generator_args)

In [127]:
1290 + 184

1474

In [128]:
a = os.listdir('../data/FGADR-Seg-set_Release/Seg-set/HardExudate_Masks/')
a.sort()
len(a[:1474])

1474

In [111]:
len(tr_eyepacks_gen.data_paths)

500

In [42]:
for input, outputs in tr_eyepacks_gen:
    break

In [43]:
print(input.shape)
print(outputs[0].shape)
print(outputs[1].shape)
print(outputs[2].shape)
print(outputs[3].shape)

(4, 512, 512, 1)
(4, 512, 512, 1)
(4, 512, 512, 1)
(4, 512, 512, 1)
(4, 512, 512, 1)


In [46]:
preds = model(input)
# 이미지 4개 예측하는데 23초나 걸리네...? , CPU라서 그런가보다

print(len(preds))
print(preds[0].shape)
print(preds[1].shape)
print(preds[2].shape)
print(preds[3].shape)
print(preds[4].shape)

5
(4, 512, 512, 1)
(4, 512, 512, 1)
(4, 512, 512, 1)
(4, 512, 512, 1)
(4, 512, 512, 1)


In [45]:
preds = model(input, only_recons=True)

print(len(preds))
print(preds[0].shape)

1
(4, 512, 512, 1)


In [89]:
import numpy as np

def dice_loss(inputs, targets, smooth = 1.):
    inputs = np.array(inputs)
    targets = np.array(targets)
    
    dice_losses = []
    for input, target in zip(inputs, targets): 
        input_flat = input.flatten()
        target_flat = target.flatten()
        
        intersection = np.sum(input_flat * target_flat)
        dice_coef = (2. * intersection + smooth) / (np.sum(input_flat) + np.sum(target_flat) + smooth)

        dice_losses.append(1. - dice_coef)
        
    return np.mean(dice_losses)
    
def mean_square_error(input_hats, inputs):
    mses = []
    for input_hat, input in zip(input_hats, inputs):
        mses.append(tf.reduce_mean(tf.square(input_hat - input)).numpy())
    return np.mean(mses)

In [57]:
dice_loss(preds[1], outputs[1])

0.9938407298426654

In [90]:
mean_square_error(preds[1], outputs[1])

0.43095237

# Classification 짜기

In [14]:
import tensorflow as tf
from tensorflow import keras

class ConvBlock(tf.keras.layers.Layer):
    def __init__(self, n_filters):
        super(ConvBlock, self).__init__()
        self.filters = n_filters
        self.conv = keras.layers.Conv2D(n_filters, 3, padding='same')
        self.bn = keras.layers.BatchNormalization()
        self.relu = keras.layers.Activation('relu') 
    
    def call(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        
        return x


class UpsampleBlock(tf.keras.layers.Layer):
    def __init__(self, n_filters):
        super(UpsampleBlock, self).__init__()
        self.filters = n_filters
        self.conv_T = keras.layers.Conv2DTranspose(n_filters, (2, 2), strides=2, padding='same')
        self.conv_block_1 = ConvBlock(n_filters)
        self.concat = keras.layers.Concatenate()
        self.conv_block_2 = ConvBlock(n_filters)

    def call(self, x, skip):
        x = self.conv_T(x)
        x = self.conv_block_1(x)
        x = self.concat([x, skip])
        x = self.conv_block_2(x)

        return x   



class EncoderBlock(tf.keras.layers.Layer):
    def __init__(self, filters):
        super(EncoderBlock, self).__init__()
        self.filters = filters # filters = [n_filters]
        # [64, 128, 256, 512, 1024]
        self.conv_blocks_0_1 = ConvBlock(self.filters[0])
        self.conv_blocks_0_2 = ConvBlock(self.filters[0])
        self.conv_blocks_1_1 = ConvBlock(self.filters[1])
        self.conv_blocks_1_2 = ConvBlock(self.filters[1])
        self.conv_blocks_2_1 = ConvBlock(self.filters[2])
        self.conv_blocks_2_2 = ConvBlock(self.filters[2])
        self.conv_blocks_3_1 = ConvBlock(self.filters[3])
        self.conv_blocks_3_2 = ConvBlock(self.filters[3])
        self.conv_blocks_4_1 = ConvBlock(self.filters[4])
        self.conv_blocks_4_2 = ConvBlock(self.filters[4])
        
        self.max_poolings_1 = keras.layers.MaxPooling2D(2)
        self.max_poolings_2 = keras.layers.MaxPooling2D(2)
        self.max_poolings_3 = keras.layers.MaxPooling2D(2)
        self.max_poolings_4 = keras.layers.MaxPooling2D(2)
        
    def call(self, x):
        skips = []
        
        x = self.conv_blocks_0_1(x)
        x = self.conv_blocks_0_2(x)
        skips.append(x)
        x = self.max_poolings_1(x)
        
        x = self.conv_blocks_1_1(x)
        x = self.conv_blocks_1_2(x)
        skips.append(x)
        x = self.max_poolings_2(x)
        
        x = self.conv_blocks_2_1(x)
        x = self.conv_blocks_2_2(x)
        skips.append(x)
        x = self.max_poolings_3(x)
        
        x = self.conv_blocks_3_1(x)
        x = self.conv_blocks_3_2(x)
        skips.append(x)
        x = self.max_poolings_4(x)
        
        x = self.conv_blocks_4_1(x)
        x = self.conv_blocks_4_2(x)

        return x, skips


class DecoderBlock(tf.keras.layers.Layer):
    def __init__(self, filters, is_recons=False, input_channel=3):
        super(DecoderBlock, self).__init__()
        self.filters = filters # filters = [n_filters]
        # enc_filters = [64, 128, 256, 512, 1024]
        # dec_filters = [512, 256, 128, 64]
        self.upsample_blocks_0 = UpsampleBlock(self.filters[0])
        self.upsample_blocks_1 = UpsampleBlock(self.filters[1])
        self.upsample_blocks_2 = UpsampleBlock(self.filters[2])
        self.upsample_blocks_3 = UpsampleBlock(self.filters[3])
        
        self.last_conv = ConvBlock(2)
        if is_recons:
            self.last_block = keras.layers.Conv2D(filters=input_channel, 
                                                  kernel_size=1, 
                                                  padding='same', 
                                                  activation='linear')
        else:
            self.last_block = keras.layers.Conv2D(filters=1, 
                                                  kernel_size=1, 
                                                  padding='same', 
                                                  activation='sigmoid')
        

    def call(self, x, skips):
        x = self.upsample_blocks_0(x, skips[0])
        x = self.upsample_blocks_1(x, skips[1])
        x = self.upsample_blocks_2(x, skips[2])
        x = self.upsample_blocks_3(x, skips[3])
        
        x = self.last_conv(x)
        x = self.last_block(x)
        
        return x

In [15]:
class UpsampleBlock(tf.keras.layers.Layer):
    def __init__(self, n_filters):
        super(UpsampleBlock, self).__init__()
        self.filters = n_filters
        self.conv_T = keras.layers.Conv2DTranspose(n_filters, (2, 2), strides=2, padding='same')
        self.conv_block_1 = ConvBlock(n_filters)
        self.concat = keras.layers.Concatenate()
        self.conv_block_2 = ConvBlock(n_filters)

    def call(self, x, skip):
        x = self.conv_T(x)
        x = self.conv_block_1(x)
        x = self.concat([x, skip])
        x = self.conv_block_2(x)

        return x   

class ClsBlock(tf.keras.layers.Layer):
    def __init__(self, filters):
        super(ClsBlock, self).__init__()
        self.filters = filters # filters = [n_filters]
        # [64, 128, 256, 512, 1024]
        self.conv_blocks_0_1 = ConvBlock(self.filters[0])
        self.conv_blocks_0_2 = ConvBlock(self.filters[0])
        self.conv_blocks_1_1 = ConvBlock(self.filters[1])
        self.conv_blocks_1_2 = ConvBlock(self.filters[1])
        self.conv_blocks_2_1 = ConvBlock(self.filters[2])
        self.conv_blocks_2_2 = ConvBlock(self.filters[2])
        self.conv_blocks_3_1 = ConvBlock(self.filters[3])
        self.conv_blocks_3_2 = ConvBlock(self.filters[3])
        self.conv_blocks_4_1 = ConvBlock(self.filters[4])
        self.conv_blocks_4_2 = ConvBlock(self.filters[4])
        
        self.max_poolings_1 = keras.layers.MaxPooling2D(2)
        self.max_poolings_2 = keras.layers.MaxPooling2D(2)
        self.max_poolings_3 = keras.layers.MaxPooling2D(2)
        self.max_poolings_4 = keras.layers.MaxPooling2D(2)
        
        self.concat_0 = keras.layers.Concatenate()
        self.concat_1 = keras.layers.Concatenate()
        self.concat_2 = keras.layers.Concatenate()
        self.concat_3 = keras.layers.Concatenate()
        
        self.gap = keras.layers.GlobalAveragePooling2D()
        
        self.dense = keras.layers.Dense(1, activation='sigmoid')
        
    def call(self, x, skips):         
        x = self.conv_blocks_0_1(x)
        x = self.concat_0([x, skips[0]])
        x = self.conv_blocks_0_2(x) 
        x = self.max_poolings_1(x)
        
        x = self.conv_blocks_1_1(x)
        x = self.concat_1([x, skips[1]])
        x = self.conv_blocks_1_2(x)
        x = self.max_poolings_2(x)
        
        x = self.conv_blocks_2_1(x)
        x = self.concat_2([x, skips[2]])
        x = self.conv_blocks_2_2(x)
        x = self.max_poolings_3(x)
        
        x = self.conv_blocks_3_1(x)
        x = self.concat_3([x, skips[3]])
        x = self.conv_blocks_3_2(x)
        x = self.max_poolings_4(x)
        
        x = self.conv_blocks_4_1(x)
        x = self.conv_blocks_4_2(x)
        
        x = self.gap(x)
        
        x = self.dense(x)

        return x

In [22]:
class SnC_Unet(tf.keras.Model):
    def __init__(self, enc_filters, dec_filters, input_channel):
        super(SnC_Unet, self).__init__()

        self.enc_filters = enc_filters
        self.dec_filters = dec_filters

        # Encoder
        self.encoder = EncoderBlock(self.enc_filters)

        # Decoder
        # 5종류의 decoder가 있음
        # reconstruction
        # HardExudate, Hemohedge, Microane, SoftExudates
         # self.HardExudate = DecoderBlock(self.dec_filters)
        # self.Hemohedge = DecoderBlock(self.dec_filters)
        # self.Microane = DecoderBlock(self.dec_filters)
        # self.SoftExudates = DecoderBlock(self.dec_filters)
        
        # 복원하기 위한 branch
        # self.reconstruction = DecoderBlock(self.dec_filters, is_recons=True, input_channel=input_channel)
        
        # 하나의 마스크로 합쳐서 예측
        # self.decoder= DecoderBlock(self.dec_filters)
        
        # clsblock
        self.clsblock = ClsBlock(self.enc_filters)
       

    def call(self, inputs, only_recons=False):
        # Encoder
        x, skips = self.encoder(inputs)
        
        # Classification
        cls_pred = self.clsblock(inputs, skips)
        
        return [cls_pred]
        # # Decoder
        # input_hat = self.reconstruction(x, skips[::-1])
        
        # reconstruction만 학습할 때 구분
        # if only_recons:     
        #     return [input_hat]
        # else:
        #     # ex = self.HardExudate(x, skips[::-1])
        #     # he = self.Hemohedge(x, skips[::-1])
        #     # ma = self.Microane(x, skips[::-1])
        #     # se = self.SoftExudates(x, skips[::-1])
            
        #     mask_hat = self.decoder(x, skips[::-1])
            
        #     return [input_hat, mask_hat]

In [23]:
model = SnC_Unet(enc_filters=[64, 128, 256, 512, 1024], dec_filters=[512, 256, 64, 32], input_channel=3)
model.build((None, 512, 512, 1))

In [7]:
import os

os.listdir('../data/FGADR-Seg-set_Release/Seg-set/')

['.DS_Store',
 'DR_Seg_Grading_Label.csv',
 'HardExudate_Masks',
 'Hemohedge_Masks',
 'IRMA_Masks',
 'Microaneurysms_Masks',
 'Neovascularization_Masks',
 'Original_Images',
 'SoftExudate_Masks']

In [28]:
import pandas as pd

label_df = pd.read_csv('../data/FGADR-Seg-set_Release/Seg-set/DR_Seg_Grading_Label.csv', header=None)
label_df = label_df.rename(columns={0:"file_name"})


In [35]:
label_df

,file_name,1
0,0000_1.png,2
1,0001_2.png,3
2,0002_1.png,2
3,0003_3.png,4
4,0004_1.png,2
...,...,...
1837,1837_3.png,4
1838,1838_3.png,3
1839,1839_1.png,2
1840,1840_1.png,4


# DenseNet 구현하기

In [2]:
from tensorflow import keras

In [ ]:
img_size = (512, 512)

inputs = keras.Input(shape=img_size + (1,))



In [ ]:
def DenseNet(x):
    # input = 224 x 224 x 3
    k = 32  # Grow Rate
    compression = 0.5   # compression factor

    # 1. Convolution
    x = layers.Conv2D(k * 2, (7, 7), strides=2, padding='same', input_shape=(224, 224, 3))(x)    # 112x112x64
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # 2. Pooling
    x = layers.MaxPool2D((3, 3), 2, padding='same')(x)  # 56x56x64

    # 3. Dense Block (1)
    for i in range(6) :
        x_l = layers.Conv2D(k * 4, (1, 1), strides=1, padding='same')(x)    # 56x56x128
        x_l = layers.BatchNormalization()(x_l)
        x_l = layers.Activation('relu')(x_l)

        x_l = layers.Conv2D(k, (3, 3), strides=1, padding='same')(x_l)  # 56x56x32
        x_l = layers.BatchNormalization()(x_l)
        x_l = layers.Activation('relu')(x_l)

        x = layers.Concatenate()([x, x_l])  # 96 -> 128 -> 160 -> 192 -> 224 -> 256

    # 4. Transition Layer (1)
    current_shape = int(x.shape[-1]) # 56x56x256
    x = layers.Conv2D(int(current_shape * compression), (1, 1), strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.AveragePooling2D((2, 2), strides=2, padding='same')(x)   # 28x28

    # 5. Dense Block (2)
    for i in range(12) :
        x_l = layers.Conv2D(k * 4, (1, 1), strides=1, padding='same')(x)
        x_l = layers.BatchNormalization()(x_l)
        x_l = layers.Activation('relu')(x_l)

        x_l = layers.Conv2D(k, (3, 3), strides=1, padding='same')(x_l)
        x_l = layers.BatchNormalization()(x_l)
        x_l = layers.Activation('relu')(x_l)

        x = layers.Concatenate()([x, x_l])

    # 6. Transition Layer (2)
    current_shape = int(x.shape[-1])
    x = layers.Conv2D(int(current_shape * compression), (1, 1), strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.AveragePooling2D((2, 2), strides=2, padding='same')(x)   # 14x14

    # 7. Dense Block (3)
    for i in range(24) :
        x_l = layers.Conv2D(k * 4, (1, 1), strides=1, padding='same')(x)
        x_l = layers.BatchNormalization()(x_l)
        x_l = layers.Activation('relu')(x_l)

        x_l = layers.Conv2D(k, (3, 3), strides=1, padding='same')(x_l)
        x_l = layers.BatchNormalization()(x_l)
        x_l = layers.Activation('relu')(x_l)

        x = layers.Concatenate()([x, x_l])

    # 8. Transition Layer (3)
    current_shape = int(x.shape[-1])
    x = layers.Conv2D(int(current_shape * compression), (1, 1), strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.AveragePooling2D((2, 2), strides=2, padding='same')(x)   # 7x7

    # 9. Dense Block (4)
    for i in range(16) :
        x_l = layers.Conv2D(k * 4, (1, 1), strides=1, padding='same')(x)
        x_l = layers.BatchNormalization()(x_l)
        x_l = layers.Activation('relu')(x_l)

        x_l = layers.Conv2D(k, (3, 3), strides=1, padding='same')(x_l)
        x_l = layers.BatchNormalization()(x_l)
        x_l = layers.Activation('relu')(x_l)

        x = layers.Concatenate()([x, x_l])

    # 10. Classification Layer
    x = layers.GlobalAveragePooling2D()(x)
    # classes = 2 (softmax)
    x = layers.Dense(2, activation='softmax')(x)

    return x
출처: https://beginnerdeveloper-lit.tistory.com/162 [초보 개발자의 이야기, 릿허브:티스토리]